In [6]:
import numpy as np
from simulation.experiment import ForwardExperiment1D
from utility.distributions import spike, ricker, gaussian, raised_cosine, sinc, homogeneous, exponential, polynomial
from utility.plotting import plot_uv, plot_state, plot_medium, plot_heatmap
import plotly.express as px

In [7]:
# Create experiment
experiment = ForwardExperiment1D()

# Set parameters 1
dx = 1
nx = 15
dt = 0.0001
nt = 15
order = 1
bcs = {'left': 'DBC', 'right': 'DBC'}
mu = np.concatenate([homogeneous(3e10, nx+1)])
rho = np.concatenate([homogeneous(3000, nx)])
u = np.concatenate([ricker(1, nx, int(nx/2), sigma=3)])
v = np.concatenate([homogeneous(0, nx)])
backend = {}

parameters = {'dx': dx, 'nx': nx, 'dt': dt, 'nt': nt, 'order': order, 'bcs': bcs,
              'mu': mu, 'rho': rho, 'u': u, 'v': v, 'backend': backend}

experiment.add_solver('ode', **parameters)
experiment.add_solver('local', **parameters)
results = experiment.run()

Jobs completed: 0 | 5
Jobs completed: 0 | 5
Jobs completed: 0 | 5
Jobs completed: 0 | 5
Jobs completed: 0 | 5
Jobs completed: 0 | 5
Jobs completed: 0 | 5
Jobs completed: 0 | 5
Jobs completed: 0 | 5
Jobs completed: 1 | 5
Jobs completed: 1 | 5
Jobs completed: 1 | 5
Jobs completed: 1 | 5
Jobs completed: 1 | 5
Jobs completed: 1 | 5
Jobs completed: 1 | 5
Jobs completed: 1 | 5
Jobs completed: 3 | 5
Jobs completed: 5 | 5


In [8]:
idx = 4
plot_uv(results[0]['field']['u'][idx], results[0]['field']['v'][idx], results[0]['times'][idx], plot_v=True).show(backend='svg')

In [9]:
plot_uv(results[1]['field']['u'][idx], results[1]['field']['v'][idx], results[1]['times'][idx], plot_v=True).show(backend='svg')